In [21]:
import ipywidgets as widgets
import datetime as dt
import pandas as pd
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]


In [22]:
# Thanks to hatyan https://github.com/Deltares/hatyan for the core of this example
url_prefix="http://uhslc.soest.hawaii.edu:80/opendap/rqds/global/hourly" 
station_urls=["h825a.nc","h822a.nc"]
station_names=["Cuxhaven","Brest"] #http://uhslc.soest.hawaii.edu/data/

In [ ]:
ts_meas=[]
ts_names=[]
print("Downloading data per location:")
for i in np.arange(len(station_urls)):
   ts_data = nc.Dataset(url_prefix+"/"+station_urls[i])
   ts_name = station_names[i]
   ts_names.append(ts_name)
   print(nc.chartostring(ts_data['station_name'][:])[0]) #check name
   #ts_values=ts_data["sea_level"][0,:]/1000 #when you get server errors, then use this instead
   #ts_values=ts_values[-1000:]
   ts_values=ts_data["sea_level"][0,-1000:]/1000
   #ts_times=ts_data_times = nc.num2date(ts_data['time'][:],units=ts_data['time'].units, only_use_cftime_datetimes=False)
   #ts_times=ts_times[-1000:]
   ts_times=ts_data_times = nc.num2date(ts_data['time'][-1000:],units=ts_data['time'].units, only_use_cftime_datetimes=False)
   ts_meas.append(pd.DataFrame({'values':ts_values},index=ts_times))
print("Downloads finished")

In [25]:
# selection of stations with a dropdown list
dd1=widgets.Dropdown(
    options=ts_names,
    value=ts_names[0],
    description='Location',
)
display(dd1)

# create output widget for a plot
out_plot = widgets.Output(layout = widgets.Layout(height='300px'))
 
def update_plot(change):
    choice=dd1.value #or change["new"]
    with out_plot:
        iplot=ts_names.index(choice)
        plt.plot(ts_meas[iplot],color="C0")
        #plt.title(ts_names[iplot])
        plt.title(choice)
        out_plot.clear_output()
        plt.show()

dd1.observe(update_plot,names="value")
display(out_plot)
update_plot(1)


Dropdown(description='Location', options=('Cuxhaven', 'Brest'), value='Cuxhaven')

Output(layout=Layout(height='300px'))

In [ ]:
#issue on laptop with ubuntu
# ts_data = nc.Dataset(url_prefix+"/"+station_urls[0])
# ts_values=ts_data["sea_level"][0,-1000:]/1000 #indexing fails here, but works on other systems
# ts_values